In [1]:
import cv2, os
import numpy as np
import nibabel as nib

In [ ]:
for img_name in os.listdir("test_images/pred_jpgs_new1/"):
    img_path = os.path.join("test_images/pred_jpgs_new1/", img_name)
    
    temp = os.path.splitext(img_name)[0]
    
    img_gray = cv2.imread(img_path, 0)
    img_rgb = cv2.imread(img_path)
    
    img_gray_thresh = img_gray.copy()
    
    #thresholding to generate mask
    for row in range(img_gray.shape[0]):
        for col in range(img_gray.shape[1]):
            if img_gray[row][col] > 190:
                img_gray_thresh[row][col] = 0
            else:
                img_gray_thresh[row][col] = 255
                
    cv2.imwrite("test_images/img_gray_thresh_new2/%s.png", img_gray_thresh)
    
    #median filter
    img_med = cv2.medianBlur(img_gray_thresh, 15, 0  )
    cv2.imwrite("test_images/img_med_new2/%s.png"%temp, img_med)
    
    #overlaying generatared mask
    final_img = cv2.bitwise_and(img_rgb, img_rgb, mask=img_med)
    cv2.imwrite("test_images/final_rgb_new2/%s.png"%temp, final_img)
    

In [ ]:
#################################################

In [ ]:
#convreting rgb_images predictions to rgb_3D Stack
#gray_images 3D Stack -> tifffile library, extensions .tif
                     # -> grayimage
    
#rgb_imges to 3D Stack - >nibabel library, extension .nii
                    # -> rgb images

In [3]:
#sort the images from 0 to n in the correct order
img_list = []
path_in = 'test_images/final_rgb_new1/'
for i in range(500):
    
    img_file = cv2.imread(path_in+"%s.png"%i)
    
    img_list.append(img_file)

img_np = np.array(img_list)

In [ ]:
img_np.shape

In [5]:

# ras_pos is a 4-d numpy array, with the last dim holding RGB
ras_pos = img_np

shape_3d = ras_pos.shape[0:3]
rgb_dtype = np.dtype([('R', 'u1'), ('G', 'u1'), ('B', 'u1')])
ras_pos = ras_pos.copy().view(dtype=rgb_dtype).reshape(shape_3d) # copy used to force fresh internal structure
ni_img = nib.Nifti1Image(ras_pos, np.eye(4))
nib.save(ni_img, 'test_images/stack/rgbStack_final_new1.nii') #path to our directory where we want to save the stack